<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We use these libraries to get financial data, analyze numbers, handle dates, model options, and make clear charts. These tools give us everything we need for pricing and visualizing financial instruments.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from QuantLib import *

We’re bringing in tools for charting, data analysis, data collection, and financial modeling. This combination helps us work with price data, build and price options, and show the results visually. Bringing them together makes our workflow smoother and more reliable.

## Retrieve and prepare financial data

We collect the most recent year of prices for a specific stock, then prepare key market data needed for our analysis.

In [ ]:
ticker = "AAPL"
data = yf.download(ticker, period="1y", multi_level_index=False)
spot_price = data.Close.iat[-1]
risk_free_rate = 0.04
dividend_yield = 0.006
vol_annual = np.std(np.log(data["Close"] / data["Close"].shift(1)).dropna()) * np.sqrt(
    252
)

We pick a stock and download its daily price history for the past year. We then pick out the latest closing price. For analysis, we assume fixed values for interest and dividends, and measure typical daily price swings to estimate how much the price usually moves in a year. This gives us a solid starting point for pricing options.

## Define option contracts and set up engine

We set the date for valuation, pick an expiry, and lay out a range of possible payouts for different option strike prices. We also build the financial model needed to estimate option values.

In [ ]:
valuation_date = Date.todaysDate()
Settings.instance().evaluationDate = valuation_date
expiry_days = 60
expiry_date = TARGET().advance(valuation_date, Period(expiry_days, Days))
maturity = Actual365Fixed().yearFraction(valuation_date, expiry_date)
option_type = Option.Call

In [ ]:
strikes = np.linspace(spot_price * 0.8, spot_price * 1.2, 61)
payoffs = [PlainVanillaPayoff(option_type, k) for k in strikes]
exercise = AmericanExercise(valuation_date, expiry_date)
option_list = [VanillaOption(payoff, exercise) for payoff in payoffs]

In [ ]:
spot_handle = QuoteHandle(SimpleQuote(spot_price))
flat_ts = YieldTermStructureHandle(
    FlatForward(valuation_date, risk_free_rate, Actual365Fixed())
)
dividend_ts = YieldTermStructureHandle(
    FlatForward(valuation_date, dividend_yield, Actual365Fixed())
)
flat_vol = BlackVolTermStructureHandle(
    BlackConstantVol(valuation_date, TARGET(), vol_annual, Actual365Fixed())
)
bsm_process = BlackScholesMertonProcess(spot_handle, dividend_ts, flat_ts, flat_vol)

In [ ]:
for opt in option_list:
    engine = BinomialVanillaEngine(bsm_process, "crr", 1000)
    opt.setPricingEngine(engine)

We establish today’s date for analysis, set when the option expires, and create a variety of options with strikes both below and above current price. Each option has the right to be exercised anytime up to expiry. We use a standard model that blends observed prices, rates, and volatility. We then assign a pricing method to each option so it can be properly valued.

## Calculate and plot option prices

We calculate the estimated market price for each different strike, then chart the results to see how value changes across strike prices.

In [ ]:
option_prices = [opt.NPV() for opt in option_list]

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(strikes, option_prices, label="American Option Price")
plt.scatter(strikes, option_prices, s=12)
plt.title(f"{ticker} American {Option.Call} Option Price vs Strike")
plt.xlabel("Strike Price")
plt.ylabel("Option Price (60 Days to Expiration)")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

We run the pricing model for each option and collect their values. Then, we build a chart with strikes on the x-axis and the corresponding option prices on the y-axis. By seeing this in a single chart, we get an intuitive sense of how the option’s price responds to different strike prices as the expiry date approaches. This makes it easy to spot patterns or key price levels.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.